# Cognitive NPL Resolution Engine

# Install & Import

In [2]:
import os
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Extract, Transform, Load (ETL)

### Config & settings up

In [4]:
CONFIG = {
    "data_paths": {
        "raw": "../data/01-raw/",
        "processed": "../data/02-preprocessed/"
    },
    "files": {
        "sba_2010_2019": "foia-7a-fy2010-fy2019-as-of-251231.csv",
        "sba_2020_present": "foia-7a-fy2020-present-as-of-251231.csv",
        "bea_gdp_long": "BEA_GDP_2017-2024.csv", 
        "bea_gdp_recent": "BEA_GDP_2023-2025.csv", 
        "dprime": "DPRIME.csv",
        "unrate": "UNRATE.csv"
    },
    "schema": {
        "sba_date_col": "approvaldate",
        "sba_target_col": "loanstatus",
        "sba_industry_col": "naicscode"
    },
   
    "naics_mapper": {
        '7225': 'Food services and drinking places', # Restaurants
        '7223': 'Food services and drinking places', # Special food services
        '7211': 'Accommodation', # Hotels
        '7139': 'Amusements, gambling, and recreation industries', # Fitness Centers
        '4841': 'Truck transportation', # Trucking
        '4842': 'Truck transportation', 
        '2361': 'Construction', # Residential Remodelers
        '2362': 'Construction', 
        '238': 'Construction', # Specialty Trade (Plumbing, Electrical falls here)
        '2381': 'Construction', 
        '2382': 'Construction', 
        '2383': 'Construction', 
        '2389': 'Construction', 
        '5617': 'Administrative and support services', # Landscaping
        '8111': 'Other services, except government', # Auto Repair
        '8121': 'Other services, except government', # Beauty Salons
        '6244': 'Social assistance', # Child Day Care
        '5242': 'Insurance carriers and related activities', # Agencies
        '6211': 'Ambulatory health care services', # Physicians
        '6212': 'Ambulatory health care services', # Dentists
        '6213': 'Ambulatory health care services', # Other Health
        '4471': 'Other retail', # Gas Stations (BEA lumps this)
        '4453': 'Food and beverage stores', # Liquor Stores
        
        '111': 'Farms', 
        '112': 'Farms',
        '113': 'Forestry, fishing, and related activities',
        '114': 'Forestry, fishing, and related activities',
        '115': 'Forestry, fishing, and related activities',
        '211': 'Oil and gas extraction',
        '212': 'Mining, except oil and gas',
        '213': 'Support activities for mining',
        '221': 'Utilities',
        '23': 'Construction',
        
        # Manufacturing
        '321': 'Wood products',
        '327': 'Nonmetallic mineral products',
        '331': 'Primary metals',
        '332': 'Fabricated metal products',
        '333': 'Machinery',
        '334': 'Computer and electronic products',
        '335': 'Electrical equipment, appliances, and components',
        '336': 'Motor vehicles, bodies and trailers, and parts',
        '337': 'Furniture and related products',
        '339': 'Miscellaneous manufacturing',
        '311': 'Food and beverage and tobacco products',
        '312': 'Food and beverage and tobacco products',
        '313': 'Textile mills and textile product mills',
        '314': 'Textile mills and textile product mills',
        '315': 'Apparel and leather and allied products',
        '316': 'Apparel and leather and allied products',
        '322': 'Paper products',
        '323': 'Printing and related support activities',
        '324': 'Petroleum and coal products',
        '325': 'Chemical products',
        '326': 'Plastics and rubber products',

        # Wholesale & Retail
        '423': 'Wholesale trade', '424': 'Wholesale trade', '425': 'Wholesale trade',
        '441': 'Motor vehicle and parts dealers',
        '445': 'Food and beverage stores',
        '452': 'General merchandise stores',
        '44': 'Retail trade', '45': 'Retail trade', # Fallback

        # Transport
        '481': 'Air transportation',
        '482': 'Rail transportation',
        '483': 'Water transportation',
        '484': 'Truck transportation',
        '485': 'Transit and ground passenger transportation',
        '486': 'Pipeline transportation',
        '493': 'Warehousing and storage',
        '48': 'Other transportation and support activities', 
        '49': 'Other transportation and support activities',

        # Information (Updated 2022 Codes)
        '511': 'Publishing industries, except internet (includes software)',
        '512': 'Motion picture and sound recording industries',
        '515': 'Broadcasting and telecommunications',
        '517': 'Broadcasting and telecommunications',
        '518': 'Data processing, internet publishing, and other information services',
        '519': 'Data processing, internet publishing, and other information services',
        '513': 'Publishing industries, except internet (includes software)', # New NAICS
        '516': 'Data processing, internet publishing, and other information services', # New NAICS

        # Finance & Real Estate
        '521': 'Federal Reserve banks, credit intermediation, and related activities',
        '522': 'Federal Reserve banks, credit intermediation, and related activities',
        '523': 'Securities, commodity contracts, and investments',
        '524': 'Insurance carriers and related activities',
        '525': 'Funds, trusts, and other financial vehicles',
        '531': 'Other real estate', 
        '532': 'Rental and leasing services and lessors of intangible assets',
        '533': 'Rental and leasing services and lessors of intangible assets',

        # Professional & Admin
        '5411': 'Legal services',
        '5415': 'Computer systems design and related services',
        '541': 'Miscellaneous professional, scientific, and technical services', 
        '551': 'Management of companies and enterprises',
        '561': 'Administrative and support services',
        '562': 'Waste management and remediation services',

        # Edu, Health, Social
        '611': 'Educational services',
        '621': 'Ambulatory health care services',
        '622': 'Hospitals',
        '623': 'Nursing and residential care facilities',
        '624': 'Social assistance',

        # Arts & Hospitality
        '711': 'Performing arts, spectator sports, museums, and related activities',
        '712': 'Performing arts, spectator sports, museums, and related activities',
        '713': 'Amusements, gambling, and recreation industries',
        '721': 'Accommodation',
        '722': 'Food services and drinking places',

        # Other & Govt
        '811': 'Other services, except government',
        '812': 'Other services, except government',
        '813': 'Other services, except government',
        '814': 'Other services, except government',
        '92': 'General government'
    },
    
    "filter": {
        "start_year": 2019, 
        "end_year": 2025
    }
}

warnings.filterwarnings('ignore')
print("Configuration Loaded. Ready for ETL.")

Configuration Loaded. Ready for ETL.


### Extract Data

data internal :  
- **`Small Business Administration`** with range **2009-2025**

data external : 

- **`Bureau of Economic Analysis`** with range **2017-2025**
- **`Federal Reserve Economic Data`** - **`Prime Rate/Fed Funds Rate`** with range **2016-2026**
- **`Federal Reserve Economic Data`** - **`Unemployment Rate`** with range **2015-2025**

### Data Internal: SBA 2009 - 2025

In [24]:
#basic Info
sba_1 = pd.read_csv ("../data/01-raw/foia-7a-fy2010-fy2019-as-of-251231.csv")
sba_1.info()

#feature info
numerical_features = [f for f in sba_1.columns if sba_1[f].dtypes != 'O']
cat_features = [c for c in sba_1.columns if sba_1[c].dtypes == 'O']

print("Numerical Features: ", numerical_features)
print("Jumlah Numerical Features: ", len(numerical_features))
print(" ")

print("Categorical Features: ", cat_features)
print("Jumlah Categorical Features: ", len(cat_features))

#Range Date Info
sba_1 = pd.DataFrame(sba_1)
date_columns = ['asofdate', 'paidinfulldate', 'chargeoffdate']

for col in date_columns:
    sba_1[col] = pd.to_datetime(sba_1[col], errors='coerce')
    
years = set()
for col in date_columns:
    years.update(sba_1[col].dt.year.dropna().unique())

unique_years = sorted(list(years))
if unique_years:
    year_range = f"{int(unique_years[0])} - {int(unique_years[-1])}"
else:
    year_range = "Tidak ada tahun ditemukan"

print("Tahun unik:", unique_years)
print("Rentang tahun:", year_range)
sba_1.head(3)

/var/folders/5k/ryl7b4vx7nzdd92vwrlw_64r0000gn/T/ipykernel_2139/2424081864.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  sba_1 = pd.read_csv ("../data/01-raw/foia-7a-fy2010-fy2019-as-of-251231.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545751 entries, 0 to 545750
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   asofdate                    545751 non-null  object 
 1   program                     545751 non-null  object 
 2   l2locid                     545246 non-null  float64
 3   borrname                    545721 non-null  object 
 4   borrstreet                  545751 non-null  object 
 5   borrcity                    545751 non-null  object 
 6   borrstate                   545751 non-null  object 
 7   borrzip                     545751 non-null  int64  
 8   bankname                    545751 non-null  object 
 9   bankfdicnumber              506748 non-null  float64
 10  bankncuanumber              17265 non-null   float64
 11  bankstreet                  545246 non-null  object 
 12  bankcity                    545246 non-null  object 
 13  bankstate     

,asofdate,program,l2locid,borrname,borrstreet,borrcity,borrstate,borrzip,bankname,bankfdicnumber,...,businesstype,businessage,loanstatus,paidinfulldate,chargeoffdate,grosschargeoffamount,revolverstatus,jobssupported,collateralind,soldsecmrktind
0,2025-12-31,7A,59698.0,Sunil Kumar Kumria dba HUGHSON SHELL,2468 3rd Street,Hughson,CA,95326,Columbia Bank,17266.0,...,INDIVIDUAL,NaN,PIF,2017-11-30,NaT,0.0,0,5.0,Y,Y
1,2025-12-31,7A,48270.0,Ecousable Inc.,500 S. Sepulveda Blvd Ste 106,MANHATTAN BEACH,CA,90266,"JPMorgan Chase Bank, National Association",628.0,...,CORPORATION,NaN,PIF,2011-07-31,NaT,0.0,0,16.0,Y,NaN
2,2025-12-31,7A,106670.0,Juan M. Valadez DBA J.'s Valadez Masonry,105 Nikki Lane,Comfort,TX,78013,United Texas Credit Union,NaN,...,INDIVIDUAL,NaN,PIF,2015-04-30,NaT,0.0,1,15.0,N,NaN


In [51]:
unique_pairs = sba_1[['naicscode', 'naicsdescription']].drop_duplicates().dropna().sort_values(by='naicscode')

print("Pasangan Unik NAICS Code dan Description:")
for index, row in unique_pairs.iterrows():
    print(f"{row['naicscode']}: {row['naicsdescription']}")

unique_pairs.to_csv(
    '../data/02-preprocessed/unique_SBA_naics_mapping.csv', 
    index=False, 
    encoding='utf-8-sig'
)
print("\nFile 'unique_SBA_naics_mapping.csv' sudah dibuat.")

Pasangan Unik NAICS Code dan Description:
111110.0: Soybean Farming
111120.0: Oilseed (except Soybean) Farming
111130.0: Dry Pea and Bean Farming
111140.0: Wheat Farming
111150.0: Corn Farming
111160.0: Rice Farming
111191.0: Oilseed and Grain Combination
111199.0: All Other Grain Farming
111211.0: Potato Farming
111219.0: Other Vegetable (except Potato) and Melon Farming
111310.0: Orange Groves
111320.0: Citrus (except Orange) Groves
111331.0: Apple Orchards
111332.0: Grape Vineyards
111333.0: Strawberry Farming
111334.0: Berry (except Strawberry) Farming
111335.0: Tree Nut Farming
111336.0: Fruit and Tree Nut Combination
111339.0: Other Noncitrus Fruit Farming
111411.0: Mushroom Production
111419.0: Other Food Crops Grown Under Cover
111421.0: Nursery and Tree Production
111422.0: Floriculture Production
111910.0: Tobacco Farming
111920.0: Cotton Farming
111940.0: Hay Farming
111991.0: Sugar Beet Farming
111992.0: Peanut Farming
111998.0: All Other Miscellaneous Crop Farming
112111.0

### Data Internal: SBA 2019 - 2025

In [25]:
#basic info
sba_2 = pd.read_csv("../data/01-raw/foia-7a-fy2020-present-as-of-251231.csv")
sba_2.info()

#feature info
numerical_features = [f for f in sba_2.columns if sba_2[f].dtypes != 'O']
cat_features = [c for c in sba_2.columns if sba_2[c].dtypes == 'O']

print("Numerical Features: ", numerical_features)
print("Jumlah Numerical Features: ", len(numerical_features))
print(" ")

print("Categorical Features: ", cat_features)
print("Jumlah Categorical Features: ", len(cat_features))

#Range Date Info
sba_2 = pd.DataFrame(sba_2)
date_columns = ['asofdate', 'paidinfulldate', 'chargeoffdate']

for col in date_columns:
    sba_2[col] = pd.to_datetime(sba_2[col], errors='coerce')
    
years = set()
for col in date_columns:
    years.update(sba_2[col].dt.year.dropna().unique())

unique_years = sorted(list(years))
if unique_years:
    year_range = f"{int(unique_years[0])} - {int(unique_years[-1])}"
else:
    year_range = "Tidak ada tahun ditemukan"

print("Tahun unik:", unique_years)
print("Rentang tahun:", year_range)
sba_2.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357866 entries, 0 to 357865
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   asofdate                    357866 non-null  object 
 1   program                     357866 non-null  object 
 2   l2locid                     357368 non-null  float64
 3   borrname                    357864 non-null  object 
 4   borrstreet                  357866 non-null  object 
 5   borrcity                    357866 non-null  object 
 6   borrstate                   357866 non-null  object 
 7   borrzip                     357866 non-null  int64  
 8   bankname                    357866 non-null  object 
 9   bankfdicnumber              319902 non-null  float64
 10  bankncuanumber              10055 non-null   float64
 11  bankstreet                  357368 non-null  object 
 12  bankcity                    357368 non-null  object 
 13  bankstate     

,asofdate,program,l2locid,borrname,borrstreet,borrcity,borrstate,borrzip,bankname,bankfdicnumber,...,businesstype,businessage,loanstatus,paidinfulldate,chargeoffdate,grosschargeoffamount,revolverstatus,jobssupported,collateralind,soldsecmrktind
0,2025-12-31,7A,507814.0,Plaza Drive Investments LLC,36223 PLAZA DR,CATHEDRAL CITY,CA,92234,"VelocitySBA, LLC",NaN,...,CORPORATION,Existing or more than 2 years old,EXEMPT,NaT,NaT,0.0,0,3,Y,Y
1,2025-12-31,7A,33850.0,Green Mountain Corporation,19301 S Santa Fe Ave,COMPTON,CA,90221,Comerica Bank,983.0,...,CORPORATION,Existing or more than 2 years old,PIF,2022-10-31,NaT,0.0,1,0,Y,NaN
2,2025-12-31,7A,112407.0,Robert D. Thompson and Lilia A. Garcia,8605 Sovereign Row,Dallas,TX,75247,Enterprise Bank & Trust,27237.0,...,CORPORATION,Existing or more than 2 years old,PIF,2023-03-31,NaT,0.0,0,4,Y,Y


### Data External: BEA GDP Industry 2023-2025

In [30]:
bea_1 = pd.read_csv ("../data/01-raw/BEA_GDP_2023-2025.csv", sep=';')
bea_1.info()
bea_1.head(6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 1 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Value Added by Industry  110 non-null    object
dtypes: object(1)
memory usage: 1012.0+ bytes


,Value Added by Industry
0,Value Added by Industry
1,"Last Revised on: January 22, 2026"
2,"Line,,2023,2023,2023,2023,2024,2024,2024,2024,..."
3,"Line,,Q1,Q2,Q3,Q4,Q1,Q2,Q3,Q4,Q1,Q2,Q3"
4,"1,"" Gross domestic product"",27216.4,275..."
5,"2,Private industries,24186.8,24462.4,24949.5,2..."


### Data External: BEA GDP Industry 2017-2024

In [ ]:
bea_2 = pd.read_csv ("../data/01-raw/BEA_GDP_2017-2024.csv", sep=';')
bea_2.info()
bea_2.head(6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 1 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   U.Value Added by Industry  202 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


,U.Value Added by Industry
0,U.Value Added by Industry
1,"Last Revised on: September 25, 2025"
2,"Line,,2017,2018,2019,2020,2021,2022,2023,2024"
3,",,,,,,,,,"
4,"1,"" Gross domestic product"",19612.1,206..."
5,"2,Private industries,17156.3,18097.8,18909.8,1..."


### Data External : Prime Rate/Fed Fund Rate 2016-2026

In [35]:
fred_prime_rate = pd.read_csv("../data/01-raw/DPRIME.csv")
fred_prime_rate.info()
fred_prime_rate = pd.DataFrame(fred_prime_rate)
date_columns = ['observation_date']

for col in date_columns:
    fred_prime_rate[col] = pd.to_datetime(fred_prime_rate[col], errors='coerce')
    
years = set()
for col in date_columns:
    years.update(fred_prime_rate[col].dt.year.dropna().unique())

unique_years = sorted(list(years))
if unique_years:
    year_range = f"{int(unique_years[0])} - {int(unique_years[-1])}"
else:
    year_range = "Tidak ada tahun ditemukan"

print("Tahun unik:", unique_years)
print("Rentang tahun:", year_range)
fred_prime_rate.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2610 entries, 0 to 2609
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  2610 non-null   object 
 1   DPRIME            2532 non-null   float64
dtypes: float64(1), object(1)
memory usage: 40.9+ KB
Tahun unik: [np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024), np.int32(2025), np.int32(2026)]
Rentang tahun: 2016 - 2026


,observation_date,DPRIME
0,2016-02-05,3.5
1,2016-02-08,3.5
2,2016-02-09,3.5
3,2016-02-10,3.5
4,2016-02-11,3.5


### Data External : FRED Unemployment rate 2015-2025

In [36]:
fred_unemploy = pd.read_csv("../data/01-raw/UNRATE.csv")
fred_unemploy.info()
fred_unemploy = pd.DataFrame(fred_unemploy)
date_columns = ['observation_date']

for col in date_columns:
    fred_unemploy[col] = pd.to_datetime(fred_unemploy[col], errors='coerce')
    
years = set()
for col in date_columns:
    years.update(fred_unemploy[col].dt.year.dropna().unique())

unique_years = sorted(list(years))
if unique_years:
    year_range = f"{int(unique_years[0])} - {int(unique_years[-1])}"
else:
    year_range = "Tidak ada tahun ditemukan"

print("Tahun unik:", unique_years)
print("Rentang tahun:", year_range)
fred_unemploy.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  121 non-null    object 
 1   UNRATE            120 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.0+ KB
Tahun unik: [np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024), np.int32(2025)]
Rentang tahun: 2015 - 2025


,observation_date,UNRATE
0,2015-12-01,5.0
1,2016-01-01,4.8
2,2016-02-01,4.9
3,2016-03-01,5.0
4,2016-04-01,5.1


### Helpers & Utils

In [9]:
def load_and_combine_sba(config):
    path = config['data_paths']['raw']
    f1 = config['files']['sba_2010_2019']
    f2 = config['files']['sba_2020_present']
    start_yr = config['filter']['start_year']
    
    print(f"Loading SBA Data...")
    
    df1 = pd.read_csv(f"{path}{f1}", low_memory=False)
    df2 = pd.read_csv(f"{path}{f2}", low_memory=False)
    
    print(f"   -> Raw File 1 Rows: {len(df1)}")
    print(f"   -> Raw File 2 Rows: {len(df2)}")

    df_sba = pd.concat([df1, df2], axis=0, ignore_index=True)

    # DATE CONVERSION
    date_col = config['schema']['sba_date_col'] 
    df_sba[date_col] = pd.to_datetime(df_sba[date_col], errors='coerce')
    df_sba['approvalfiscalyear'] = pd.to_numeric(df_sba['approvalfiscalyear'], errors='coerce')

    # REMOVE DUPLICATES
    before_dedup = len(df_sba)
    df_sba = df_sba.drop_duplicates(subset=['borrname', 'grossapproval', 'approvaldate', 'borrzip'])
    after_dedup = len(df_sba)
    print(f"   -> Duplicates Removed: {before_dedup - after_dedup} rows")

    df_sba = df_sba[df_sba['approvalfiscalyear'] >= start_yr]
    
    min_yr = df_sba['approvalfiscalyear'].min()
    max_yr = df_sba['approvalfiscalyear'].max()
    print(f"   -> Filtering Scope ({start_yr}+): Remaining {len(df_sba)} rows")

    df_sba['bankzip'] = df_sba['bankzip'].fillna("").astype(str).str.replace(r'\.0$', '', regex=True)
    df_sba['borrzip'] = df_sba['borrzip'].fillna("").astype(str).str.replace(r'\.0$', '', regex=True)

    object_cols = ['bankstate', 'bankcity', 'borrstate', 'borrcity', 'projectstate']
    for col in object_cols:
        if col in df_sba.columns:
            df_sba[col] = df_sba[col].fillna("Unknown").astype(str)

    print("   -> Mapping Industry to BEA (Hierarchical Strategy: 4->3->2 Digit)...")
    
    df_sba['NAICS_FULL'] = df_sba['naicscode'].fillna(0).astype(int).astype(str)
    
    df_sba['NAICS_4'] = df_sba['NAICS_FULL'].str[:4]
    df_sba['NAICS_3'] = df_sba['NAICS_FULL'].str[:3]
    df_sba['NAICS_2'] = df_sba['NAICS_FULL'].str[:2]
    
    mapper = config['naics_mapper']
    
    df_sba['Industry_BEA'] = df_sba['NAICS_4'].map(mapper)
    df_sba['Industry_BEA'] = df_sba['Industry_BEA'].fillna(df_sba['NAICS_3'].map(mapper))
    df_sba['Industry_BEA'] = df_sba['Industry_BEA'].fillna(df_sba['NAICS_2'].map(mapper)) 
    df_sba = df_sba.sort_values(config['schema']['sba_date_col'])
    
    return df_sba

def process_bea_gdp(config):
    path = config['data_paths']['raw']
    files = [config['files']['bea_gdp_long'], config['files']['bea_gdp_recent']]
    
    df_combined = pd.DataFrame()
    
    print(f"Processing BEA GDP Data...")
    
    for f in files:
        full_path = f"{path}{f}"
        try:
            temp = pd.read_csv(full_path, skiprows=3, header=[0, 1])
        
            is_truly_quarterly = False
            if isinstance(temp.columns, pd.MultiIndex):
                for col in temp.columns:
                    if 'Q1' in str(col[1]) or 'Q1' in str(col[0]):
                        is_truly_quarterly = True
                        break
            
            if is_truly_quarterly:
                print(f" -> Detected Quarterly Format: {f}")
                new_cols = []
                for col in temp.columns:
                    if 'Line' in str(col[0]): new_cols.append('LineCode')
                    elif 'Unnamed' in str(col[0]): new_cols.append('Industry')
                    else: new_cols.append(f"{col[0]}-{col[1]}")
                temp.columns = new_cols
                if 'LineCode' in temp.columns: temp = temp.drop(columns=['LineCode'])
                temp = temp.loc[:, ~temp.columns.duplicated()]
                
                # Melt
                temp_melt = temp.melt(id_vars=['Industry'], var_name='Quarter_Str', value_name='GDP_Value')
                
                def parse_quarter(q_str):
                    try:
                        y, q = q_str.split('-')
                        q_map = {'Q1': '03-31', 'Q2': '06-30', 'Q3': '09-30', 'Q4': '12-31'}
                        return pd.to_datetime(f"{y}-{q_map.get(q, '12-31')}")
                    except: return pd.NaT
                
                temp_melt['date_quarter'] = temp_melt['Quarter_Str'].apply(parse_quarter)
                temp_melt['Source'] = 'HighRes'
                df_combined = pd.concat([df_combined, temp_melt.dropna(subset=['date_quarter'])])
            else:
                raise ValueError("Not MultiIndex with Q1")

        except Exception as e:
            try:
                print(f" -> Detected Annual/Simple Format: {f} (Reason: {e})")
                df_a = pd.read_csv(full_path, skiprows=3)
                if len(df_a.columns) > 1: df_a.rename(columns={df_a.columns[1]: 'Industry'}, inplace=True)
                year_cols = [c for c in df_a.columns if str(c).strip().isdigit()]
                df_a = df_a[['Industry'] + year_cols]
                df_a_melt = df_a.melt(id_vars=['Industry'], var_name='Year', value_name='GDP_Value')
                
                # UPSAMPLING
                dfs_expanded = []
                for q_end in ['03-31', '06-30', '09-30', '12-31']:
                    temp_expand = df_a_melt.copy()
                    temp_expand['date_quarter'] = pd.to_datetime(temp_expand['Year'].astype(str) + '-' + q_end)
                    dfs_expanded.append(temp_expand)
                
                df_upsampled = pd.concat(dfs_expanded)
                df_upsampled['Source'] = 'LowRes_Upsampled'
                df_combined = pd.concat([df_combined, df_upsampled])
                
            except Exception as e2:
                print(f"Warning: Failed to read {f}. {e2}")

    if not df_combined.empty:
        df_combined['Industry'] = df_combined['Industry'].str.strip()
        df_combined['Source_Rank'] = df_combined['Source'].map({'HighRes': 1, 'LowRes_Upsampled': 2})
        df_combined = df_combined.sort_values(by=['date_quarter', 'Industry', 'Source_Rank'])
        df_combined = df_combined.drop_duplicates(subset=['date_quarter', 'Industry'], keep='first')
        df_combined['GDP_Value'] = pd.to_numeric(df_combined['GDP_Value'], errors='coerce')
        
        max_bea_date = df_combined['date_quarter'].max()
        target_fill_date = pd.Timestamp('2025-12-31')
        
        if max_bea_date < target_fill_date:
            print(f"   -> Extrapolating GDP from {max_bea_date.date()} to {target_fill_date.date()}...")
            last_q_data = df_combined[df_combined['date_quarter'] == max_bea_date].copy()
            last_q_data['date_quarter'] = target_fill_date
            df_combined = pd.concat([df_combined, last_q_data], ignore_index=True)
        
        df_combined = df_combined[['date_quarter', 'Industry', 'GDP_Value']]
        print(f" -> Total BEA Rows (Merged & Filled): {len(df_combined)}")
        return df_combined
    else:
        return pd.DataFrame()

def process_fred(config, type_key):
    """Load and standardize FRED data (DPRIME or UNRATE)."""
    path = config['data_paths']['raw']
    fname = config['files'][type_key]
    
    print(f"Processing {type_key}...")
    df = pd.read_csv(f"{path}{fname}")
    df['observation_date'] = pd.to_datetime(df['observation_date'])
    
    df = df.sort_values('observation_date')
    val_col = 'DPRIME' if type_key == 'dprime' else 'UNRATE'
    
    df.rename(columns={df.columns[1]: val_col}, inplace=True)
    df[val_col] = df[val_col].ffill()
    
    return df

def etl_pipeline(config):
    """Orchestrate the Merge (The Great Join)."""
    
    df_sba = load_and_combine_sba(config)
    df_bea = process_bea_gdp(config)
    df_dprime = process_fred(config, 'dprime')
    df_unrate = process_fred(config, 'unrate')
    
    # Left Join
    print("\nStarting The Great Join...")
    
    # Merge DPRIME (Daily) using merge_asof (Nearest Backward)
    df_master = pd.merge_asof(
        df_sba, 
        df_dprime, 
        left_on=config['schema']['sba_date_col'], 
        right_on='observation_date', 
        direction='backward'
    )
    
    # Merge UNRATE (Monthly) using merge_asof (Nearest Backward)
    df_master = pd.merge_asof(
        df_master, 
        df_unrate, 
        left_on=config['schema']['sba_date_col'], 
        right_on='observation_date', 
        direction='backward',
        suffixes=('', '_unrate')
    )
    
    # Merge BEA GDP (Quarterly + Industry)
    df_master['Quarter_Date'] = df_master[config['schema']['sba_date_col']] + pd.offsets.QuarterEnd(0)
    df_bea_renamed = df_bea.rename(columns={'date_quarter': 'Quarter_Date', 'Industry': 'Industry_BEA'})
    
    # Left Join on [Industry, Quarter_Date]
    df_master = pd.merge(
        df_master,
        df_bea_renamed[['Quarter_Date', 'Industry_BEA', 'GDP_Value']],
        on=['Quarter_Date', 'Industry_BEA'],
        how='left'
    )
    
    return df_master

### ETL Call

In [10]:
# Execution
df_final = etl_pipeline(CONFIG)

# Audit
print("\n[Audit After Merge..]")
target_col = 'GDP_Value' 
missing_gdp = df_final[target_col].isna().sum()
missing_prime = df_final['DPRIME'].isna().sum()

print(f" Missing GDP Data: {missing_gdp} rows ({missing_gdp/len(df_final):.1%})")
print(f" Missing Prime Rate: {missing_prime} rows ({missing_prime/len(df_final):.1%})")

# Check Root Cause for Missing GDP
if missing_gdp > 0:
    print("\n   -> Investigating Missing GDP:")

    missing_naics = df_final[df_final['Industry_BEA'].isna()]['naicscode'].unique()
    print(f"      NAICS codes not mapped to BEA: {missing_naics[:10]} ...")
    
    min_date_sba = df_final['approvaldate'].min()
    print(f"      SBA Date Range: {min_date_sba} to {df_final['approvaldate'].max()}")

# Export 
if len(df_final) > 0:
    save_path = f"{CONFIG['data_paths']['processed']}data_master.parquet"
    
    # Save to Parquet
    df_final.to_parquet(save_path, index=False)
    print(f"\n[SUCCESS] Master Data Saved: {save_path}")
    print(f"Shape: {df_final.shape}")
else:
    print("\n[FAILED] Dataframe empty.")

Loading SBA Data...
   -> Raw File 1 Rows: 545751
   -> Raw File 2 Rows: 357866
   -> Duplicates Removed: 6970 rows
   -> Filtering Scope (2019+): Remaining 407874 rows
   -> Mapping Industry to BEA (Hierarchical Strategy: 4->3->2 Digit)...
Processing BEA GDP Data...
 -> Detected Annual/Simple Format: BEA_GDP_2017-2024.csv (Reason: Not MultiIndex with Q1)
 -> Detected Quarterly Format: BEA_GDP_2023-2025.csv
   -> Extrapolating GDP from 2025-09-30 to 2025-12-31...
 -> Total BEA Rows (Merged & Filled): 6576
Processing dprime...
Processing unrate...

Starting The Great Join...

[Audit After Merge..]
 Missing GDP Data: 40 rows (0.0%)
 Missing Prime Rate: 0 rows (0.0%)

   -> Investigating Missing GDP:
      NAICS codes not mapped to BEA: [] ...
      SBA Date Range: 2018-10-01 00:00:00 to 2025-12-31 00:00:00

[SUCCESS] Master Data Saved: ../data/02-preprocessed/data_master.parquet
Shape: (407874, 54)


## Data Harmonization Report

### 1. The Logic Behind Data Selection
In building this **`Stochastic Credit Scoring: Integrating Copula-Monte Carlo Simulations with Ensemble Learning for Systemic Risk Modeling`**, I did not want to rely solely on internal borrower history. My hypothesis is that credit risk is not just about who the borrower is, but also *where they operate economically*. Therefore, I curated three specific datasets:

*   **SBA Loan Data (Internal):** This is our "Ground Truth." It contains the loan performance (Default vs. Paid) and specific business characteristics (Location, NAICS Industry).
*   **BEA GDP by Industry (External):** I selected this to capture **Sectoral Risk**. A generic economic slump affects a "Tech Startup" differently than a "Restaurant." By linking loans to their specific industry's GDP, the model can learn sector-specific vulnerabilities.
*   **FRED Macro Indicators (External):** I included Interest Rates (DPRIME) and Unemployment (UNRATE) to capture **Systemic Risk**. These variables answer: "Is the cost of money rising?" and "Is consumer purchasing power falling?"

### 2. Strategic Focus: The 2019–2025 Horizon
I made a conscious decision to filter the dataset to the 2019–2025 period. Here is my reasoning:

*   **Relevance over Volume:** Pre-2019 data reflects an economic cycle that no longer exists. Post-pandemic consumer behaviors and inflation dynamics are structural breaks.
*   **Crisis Learning:** By including 2019 (Baseline), 2020 (Shock), and 2023-2025 (Recovery/Inflation), I ensure the model learns how borrowers behave during extreme volatility, which is crucial for stress testing.

### 3. Data Justification & Harmonization Techniques
To merge these disparate sources effectively, I had to make several methodological decisions to ensure data integrity. Here is how I justified each step:

### A. Industry Mapping (SBA to BEA)
*   **The Challenge:** SBA data is very granular (6-digit NAICS), while GDP data is aggregated. Direct matching resulted in significant data loss.
*   **My Approach:** I implemented a **Hierarchical Cascading Logic**. I first attempted to match the specific 4-digit industry. If unavailable, I looked for the 3-digit sub-sector, and finally the 2-digit major sector.
*   **Justification:** I prioritized *Precision* where possible but established a "Safety Net" to ensure **100% Coverage**. I believe it is better to have a broad sector trend than to exclude a borrower entirely due to a missing specific code.

### B. Frequency Harmonization (GDP Data)
*   **The Challenge:** The available GDP data was a mix of high-frequency (Quarterly) for recent years and lower-frequency (Annual) for historical years.
*   **My Approach:** I used **Upsampling** for the historical annual data (applying the annual value to all four quarters) while preserving the true quarterly volatility for the recent data.
*   **Justification:** This allows the model to utilize the full historical timeline without discarding valuable training data from the pandemic era.

### C. Addressing Reporting Lags (The "Q4 2025" Gap)
*   **The Challenge:** Official government GDP data has a reporting lag. Loans disbursed in Q4 2025 (Oct-Dec) effectively had "future" dates compared to the latest available economic report.
*   **My Approach:** I applied a **Forward-Fill (Naïve Forecast)** strategy, assuming the economic conditions of Q3 persist into Q4 until official revisions are released.
*   **Justification:** I chose this over dropping the data because excluding the most recent loans would introduce **Survivorship Bias** and blind the model to the latest lending trends.

### Load Data

In [7]:
data_master = pd.read_parquet("../data/02-preprocessed/data_master.parquet")
data_master

,asofdate,program,l2locid,borrname,borrstreet,borrcity,borrstate,borrzip,bankname,bankfdicnumber,...,NAICS_4,NAICS_3,NAICS_2,Industry_BEA,observation_date,DPRIME,observation_date_unrate,UNRATE,Quarter_Date,GDP_Value
0,12/31/2025,7A,36581.0,IYENGAR DELIVERY CORPORATION - WORCESTER,55 LYMAN ST,NORTHBOROUGH,MA,1532,Stearns Bank National Association,10988.0,...,4921,492,49,Other transportation and support activities,2018-10-01,5.25,2018-10-01,3.8,2018-12-31,133.7
1,12/31/2025,7A,57328.0,ETREND INC,14231 W. 11 Mile Road,Oak Park,MI,48237,The Huntington National Bank,6560.0,...,4236,423,42,Wholesale trade,2018-10-01,5.25,2018-10-01,3.8,2018-12-31,1222.1
2,12/31/2025,7A,317954.0,bill berger llc,2420 FLORIAN ST,HAMTRAMCK,MI,48212,"Newtek Small Business Finance, Inc.",NaN,...,7115,711,71,"Performing arts, spectator sports, museums, an...",2018-10-01,5.25,2018-10-01,3.8,2018-12-31,135.8
3,12/31/2025,7A,21299.0,VAGABOND NATURE LLC,3500 Overland Avenue,Los Angeles,CA,90034,Midland States Bank,1040.0,...,7139,713,71,"Amusements, gambling, and recreation industries",2018-10-01,5.25,2018-10-01,3.8,2018-12-31,91.4
4,12/31/2025,7A,97605.0,Springridge Market LLC,9504 Springridge Texas Line Ro,Keithville,LA,71047,Carter FCU,NaN,...,4471,447,44,Other retail,2018-10-01,5.25,2018-10-01,3.8,2018-12-31,661.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407869,12/31/2025,7A,52905.0,Belvie Lash & Head Spa Inc,4266 32nd Avenue South,Fargo,ND,58104,First Western Bank & Trust,19123.0,...,8121,812,81,"Other services, except government",2025-12-31,6.75,2025-12-01,4.4,2025-12-31,NaN
407870,12/31/2025,7A,29599.0,Teriyaki House Inc,217 Westminster Street,Providence,RI,2903,Northeast Bank,19690.0,...,7225,722,72,Food services and drinking places,2025-12-31,6.75,2025-12-01,4.4,2025-12-31,NaN
407871,12/31/2025,7A,49874.0,J BRIX WINES LLC,5611 Palmer Way,CARLSBAD,CA,92008,"Newtek Bank, National Association",18734.0,...,4453,445,44,Food and beverage stores,2025-12-31,6.75,2025-12-01,4.4,2025-12-31,NaN
407872,12/31/2025,7A,77468.0,Nuvel Investments LLC,20301 GRANDE OAK SHOPPES BLVD,ESTERO,FL,33928,HomeTrust Bank,27677.0,...,7225,722,72,Food services and drinking places,2025-12-31,6.75,2025-12-01,4.4,2025-12-31,NaN


## Sanity Check's

In [11]:
numerical_features = [f for f in data_master.columns if data_master[f].dtypes != 'O']
cat_features = [c for c in data_master.columns if data_master[c].dtypes == 'O']

print("Numerical Features: ", numerical_features)
print("Jumlah Numerical Features: ", len(numerical_features))
print(" ")

print("Categorical Features: ", cat_features)
print("Jumlah Categorical Features: ", len(cat_features))

data_master = pd.DataFrame(data_master)
date_columns = ['asofdate', 'paidinfulldate', 'chargeoffdate']

for col in date_columns:
    data_master[col] = pd.to_datetime(data_master[col], errors='coerce')
    
years = set()
for col in date_columns:
    years.update(data_master[col].dt.year.dropna().unique())

unique_years = sorted(list(years))
if unique_years:
    year_range = f"{int(unique_years[0])} - {int(unique_years[-1])}"
else:
    year_range = "Tidak ada tahun ditemukan"

print("Tahun unik:", unique_years)
print("Rentang tahun:", year_range)

data_master.info()

Numerical Features:  ['l2locid', 'bankfdicnumber', 'bankncuanumber', 'grossapproval', 'sbaguaranteedapproval', 'approvaldate', 'approvalfiscalyear', 'initialinterestrate', 'terminmonths', 'naicscode', 'congressionaldistrict', 'grosschargeoffamount', 'revolverstatus', 'jobssupported', 'observation_date', 'DPRIME', 'observation_date_unrate', 'UNRATE', 'Quarter_Date', 'GDP_Value']
Jumlah Numerical Features:  20
 
Categorical Features:  ['asofdate', 'program', 'borrname', 'borrstreet', 'borrcity', 'borrstate', 'borrzip', 'bankname', 'bankstreet', 'bankcity', 'bankstate', 'bankzip', 'firstdisbursementdate', 'processingmethod', 'subprogram', 'fixedorvariableinterestind', 'naicsdescription', 'franchisecode', 'franchisename', 'projectcounty', 'projectstate', 'sbadistrictoffice', 'businesstype', 'businessage', 'loanstatus', 'paidinfulldate', 'chargeoffdate', 'collateralind', 'soldsecmrktind', 'NAICS_FULL', 'NAICS_4', 'NAICS_3', 'NAICS_2', 'Industry_BEA']
Jumlah Categorical Features:  34
Tahun u